In [1]:
#Import Libraries
import pandas as pd
import dask.dataframe as dd
from dask.distributed import Client, progress

In [2]:
filepath = '../box_downloads/LoanLevelMonthly_NEW_Core_20210201.csv'

In [3]:
#Setting client to view progress of each command
client = Client(n_workers=2, threads_per_worker=2, memory_limit='1GB')
client

Client Scheduler: tcp://127.0.0.1:57029 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 4 Memory: 2.00 GB


In [4]:
#Set datatypes to allow for better memory usage
df = dd.read_csv(filepath, dtype={'CollectionFees': 'float64',
       'CumulBK': 'float64',
       'DefaultAmount': 'float64',
       'ExtensionExecutionDate': 'object',
       'ExtensionStatus': 'object',
       'LoanAmount': 'float64',
       'MargPrinBK': 'float64',
       'PrinAdjustments': 'float64',
       'RecoveryPayments': 'float64',
       'RecoveryPrin': 'float64'})

In [5]:
#Remove all columns but ones necessary for processing and mapping to join later
df = df[['LoanID','ListingNumber','ListingCategoryName','CycleCounter','FICORange','ScorexRange']]

In [6]:
#Dedupe file based on CycleCounter field (first instance of loan file)
df = df[df.CycleCounter == 0]

In [7]:
#Compute dask file in order to actually process previous commands
df.compute()

,LoanID,ListingNumber,ListingCategoryName,CycleCounter,FICORange,ScorexRange
0,256748,2187631,Other,0,720-739,680-699
1,256812,2253012,Debt Consolidation,0,800-819,820-839
2,257165,2280474,Other,0,760-779,800-819
3,257440,2273313,Home Improvement,0,780-799,800-819
4,257515,2193175,Debt Consolidation,0,720-739,700-719
...,...,...,...,...,...,...
40798,160,1366,Not Available,0,NaN,NaN
40835,245,2231,Not Available,0,NaN,NaN
43226,318,3392,Not Available,0,NaN,NaN
43227,328,3196,Not Available,0,NaN,NaN


In [8]:
#Dask defaults to split files into .PART to save memory. Single_File parameter fixes this.
df.to_csv('dask_df_dropped_cols_v3.csv', single_file = True)

['C:/Users/P2773714/Documents/Regis/Practicum I/Prosper_scripts/dask_df_dropped_cols_v3.csv']

In [9]:
df.head()

,LoanID,ListingNumber,ListingCategoryName,CycleCounter,FICORange,ScorexRange
0,256748,2187631,Other,0,720-739,680-699
1,256812,2253012,Debt Consolidation,0,800-819,820-839
2,257165,2280474,Other,0,760-779,800-819
3,257440,2273313,Home Improvement,0,780-799,800-819
4,257515,2193175,Debt Consolidation,0,720-739,700-719


In [10]:
df.describe()

,LoanID,ListingNumber,CycleCounter
npartitions=1,,,
,float64,float64,float64
,...,...,...


In [11]:
#Compare file to Master Loan file
example = df[df.LoanID == 9422]

In [12]:
example.compute()

,LoanID,ListingNumber,ListingCategoryName,CycleCounter,FICORange,ScorexRange
44782,9422,113183,Not Available,0,NaN,580-599
